In [171]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision

# Model

In [172]:
class Autoencoder(nn.Module):
    
    def __init__(self):
        super(Autoencoder, self).__init__()
        self.encoder1 = nn.Conv2d(3, 16, 3, padding=1)
        self.encoder2 = nn.Conv2d(16, 8, 3, padding=1)
        self.encoder3 = nn.Conv2d(8, 4, 3, padding=1)
        self.encoder4 = nn.Conv2d(4, 4, 3, padding=1)
        
        self.pool = nn.MaxPool2d(2, ceil_mode=True)
        self.up1 = nn.Upsample(75, mode='nearest')
        self.up = nn.Upsample(scale_factor=2, mode='nearest')
        
        self.decoder1 = nn.Conv2d(4, 4, 3, padding=1)
        self.decoder2 = nn.Conv2d(4, 4, 3, padding=1)
        self.decoder3 = nn.Conv2d(4, 8, 3, padding=1)
        self.decoder4 = nn.Conv2d(8, 16, 3, padding=1)
        self.decoder5 = nn.Conv2d(16, 3, 3, padding=1)
        
    def forward(self, x):
        x = F.relu(self.encoder1(x))
        x = self.pool(x)
        x = F.relu(self.encoder2(x))
        x = self.pool(x)
        x = F.relu(self.encoder3(x))
        x = self.pool(x)
        x = F.relu(self.encoder4(x))
        encoder = self.pool(x)
        
        x = F.relu(self.decoder1(encoder))
        x = self.up1(x)
        x = F.relu(self.decoder2(x))
        x = self.up(x)
        x = F.relu(self.decoder3(x))
        x = self.up(x)
        x = F.relu(self.decoder4(x))
        x = self.up(x)
        decoder = F.relu(self.decoder5(x))
        
        return decoder

In [173]:
model = Autoencoder()
model

Autoencoder(
  (encoder1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (encoder2): Conv2d(16, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (encoder3): Conv2d(8, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (encoder4): Conv2d(4, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=True)
  (up1): Upsample(size=75, mode=nearest)
  (up): Upsample(scale_factor=2.0, mode=nearest)
  (decoder1): Conv2d(4, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (decoder2): Conv2d(4, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (decoder3): Conv2d(4, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (decoder4): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (decoder5): Conv2d(16, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
)

In [174]:
with torch.no_grad():
    model.eval()
    img = torch.rand(6,3,600,600)
    print(model(img).shape)

torch.Size([6, 3, 600, 600])


# Dataset

In [175]:
import os
import glob

from torch.utils.data import DataLoader, Dataset
from PIL import Image

class myDataset(Dataset):
    
    def __init__(self, image_path, transform=None):
        self.image_path = image_path
        self.transform = transform
        self.image_list = glob.glob(image_path + '/*')
        
    def __len__(self):
        
        return len(self.image_list)
    
    def __getitem__(self, idx):
        file_name = self.image_list[idx]
        
        image = Image.open(file_name)
        
        if self.transform:
            image = self.transform(image)
            
        return image, image  

def collate_fn(batch):
    return tuple(zip(*batch))

In [176]:
import torchvision.transforms as transforms

transform = transforms.Compose([
    transforms.Resize((600,600)),
    transforms.Grayscale(3),
    transforms.ToTensor()
])

train_path = r'C:\Users\gjust\Documents\Github\data\fruit\apple'
test_path = r'C:\Users\gjust\Documents\Github\data\fruit\apple'

trainset = myDataset(train_path, transform=transform)
testset = myDataset(test_path, transform=transform)

trainloader = DataLoader(trainset, batch_size=4)
testloader = DataLoader(testset, batch_size=4)

inputs, outputs = iter(trainloader).next()

In [3]:
import os
import glob

from dataset import myDataset
from model import AutoEncoder

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision

from PIL import Image
import torchvision.transforms as transforms


import argparse

# Train
def train(opt, model, dataloader, optimizer, loss_func, device):
    model.train()
    train_iter_loss = []
    print('hello')
    for i, (images, labels) in enumerate(dataloader):
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = loss_func(outputs, labels)
        loss.backward()
        optimizer.step()
        
        train_iter_loss.append(loss.item())
        
    return train_iter_loss

# Test
def test(opt, model, dataloader, loss_func, device):
    model.eval()
    val_iter_loss = []
    
    for i, (images, labels) in enumerate(dataloader):
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        loss = loss_func(outputs, labels)
        val_iter_loss.append(loss.item())
    
    return val_iter_loss
    
# HyperParameters
## LR, EPOCH, Batchsize, 
def parse_opt():
    parser = argparse.ArgumentParser(description='Train AutoEncoder Network')
    parser.add_argument('--train_path', help='training dataset', type=str)
    parser.add_argument('--val_path', help='validation dataset', type=str)

    parser.add_argument('--epoch', help='epoch', default=20, type=int)
    parser.add_argument('--train_batch_size', help='train batch_size', default=16, type=int)
    parser.add_argument('--val_batch_size', help='validation batch_size', default=8, type=int)
    parser.add_argument('--lr', help='learning rate', default=0.001, type=float)
    
    opt = parser.parse_args()
    
    return opt

def main(opt):
    
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    
    # Dataset
    print('Dataset....')
    transform = transforms.Compose([
        transforms.Resize((600,600)),
        transforms.Grayscale(3),
        transforms.ToTensor()
    ])
    
    train_set = myDataset(image_path=opt.train_path, transform=transform)
    val_set = myDataset(image_path=opt.val_path, transform=transform)
    
    train_loader = DataLoader(train_set, batch_size=opt.train_batch_size)
    val_loader = DataLoader(val_set, batch_size=opt.val_batch_size)
    
    # Model
    print('Model....')
    model = AutoEncoder()
    model.to(device)
    
    # Optimizer
    optimizer = optim.Adam(model.parameters(), lr=opt.lr)
    loss_func = nn.MSELoss()
        
    # Train
    print('Training....')
    for e in range(opt.epoch):
        print(device)
        train_iter_loss = train(opt, model, train_loader, optimizer, loss_func, device)
        print(f'Train Epoch[{e+1}/{opt.epoch}] / Loss : {sum(train_iter_loss)/len(train_loader)}')
        
        val_iter_loss = train(opt, model, val_loader, loss_func, device)
        print(f'Validation Epoch[{e+1}/{opt.epoch}] / Loss : {sum(val_iter_loss)/len(val_loader)}')
        
if __name__ == '__main__':
    opt = parse_opt()
    main(opt)

usage: ipykernel_launcher.py [-h] [--train_path TRAIN_PATH] [--val_path VAL_PATH] [--epoch EPOCH] [--train_batch_size TRAIN_BATCH_SIZE] [--val_batch_size VAL_BATCH_SIZE] [--lr LR]
ipykernel_launcher.py: error: unrecognized arguments: -f C:\Users\gjust\AppData\Roaming\jupyter\runtime\kernel-c497b08d-1a8f-49a5-8891-f0dd3862425f.json


SystemExit: 2

C:\Users\gjust\Documents\Github\Pytorch-Cookbook\pytorch\lib\site-packages\IPython\core\interactiveshell.py:3452: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


# GPU

In [177]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)
print(device)

cuda


# Optimizer

In [178]:
optimizer = optim.Adam(model.parameters(), lr=0.001)
loss_func = nn.MSELoss()
loss_func

MSELoss()

# 학습

In [180]:
EPOCH = 10
train_epoch_loss = []
val_peoch_loss = []

for e in range(EPOCH):
    
    # Train
    model.train()
    train_iter_loss = []
    for i, (images, labels) in enumerate(trainloader):
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = loss_func(outputs, labels)
        loss.backward()
        optimizer.step()
        
        train_iter_loss.append(loss.item())
        
    train_epoch_loss.append(sum(iter_loss).item())
    print(f'Train Epoch[{e+1}/{EPOCH}] / Loss : {loss}')
    
    # Validation
    
    if e+1 == 5:
        model.eval()
        val_iter_loss = []
        for i, (images, labels) in enumerate(testloader):
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            loss = loss_func(outputs, labels)
            val_iter_loss.append(loss.item())

        val_peoch_loss.append(sum(val_iter_loss).item())
        print(f'Validation Epoch[{e+1}/{EPOCH}] / Loss : {loss}')

NameError: name 'iter_loss' is not defined